<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/Finetuning_with_text_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # pretrained MedGemma
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [4]:
from google.colab import files
uploaded = files.upload()



Saving eczema_data.csv to eczema_data.csv


In [5]:
import pandas as pd
from datasets import Dataset

# Load your uploaded CSV file
data = pd.read_csv("eczema_data.csv")

# Display sample rows
print(data.head())

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(data)

# Optional: split into train/test sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


         Disease name                                               Text
0              Eczema  "I was diagnosed with eczema a few months ago,...
1              Eczema  "I've noticed these itchy, scaly patches on my...
2  Contact Dermatitis   "My hands are dry and cracked, and the skin i...
3  Contact Dermatitis   "My feet are burning and itching after my run...
4              Eczema  Eczema, also known as atopic dermatitis, prese...


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # MedGemma base model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
!pip install datasets transformers accelerate -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 32.4 MB/s eta 0:00:00


In [9]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load dataset
import pandas as pd
df = pd.read_csv("eczema_data.csv")

# Convert to HF dataset
dataset = Dataset.from_pandas(df)

# Load model & tokenizer
model_name = "google/medgemma-4b-pt"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Preprocessing function
def preprocess_function(examples):
    texts = [
        f"Disease: {d}\nDescription: {t}"
        for d, t in zip(examples["Disease name"], examples["Text"])
    ]
    tokenized = tokenizer(
        texts,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Ready to fine-tune




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [11]:
!pip install -U transformers


In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./medgemma-finetuned-eczema",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-3954448250.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'xla').